## **Setup**
Click the Play Button. No need to expand. Do not touch this. 

In [ ]:
#@title Import Files from Repo
import ipywidgets as widgets
from ipywidgets import GridspecLayout
from ipywidgets import AppLayout, Button, Layout, jslink, IntText, IntSlider
import requests
#Creates a text document using the Raw github url. This text document will be used as a .py file for imports
def import_text(text):
  url = 'https://raw.githubusercontent.com/byuccl/digital_design_colab2/master/Exercises/state_machines/files/%s' % text
  resp = requests.get(url)
  with open(text, 'wb') as f:
    f.write(resp.content)


import_text("ms_sg.py")
import_text("mc_sg.py")




from ms_sg import *
from mc_sg import *

## State Machines

Think about a vending machine. It waits for someone to approach. It then waits for you to enter the code for the snack you want. Then it takes your money from you for the snack. Then it gives you the snack. It also has to be ready in case the user cancels their choice, takes too long or messes up the payment, and then it resets and waits for new instructions.

There are 4 unique modes the vending machine can be in.
1. Waiting for a user
2. Entering a selectiion
3. Waiting for payment
4. Dispensing Food

This is called a state machine. Each mode is called a state. The circuit can remember what state it is in. When certain conditions are met, the circuit will transition to a different state. The circuit does certain actions depending on what state it is in. The two big parts of a state machine are the actions and transitions. Actions are things that happen in a state and transitions are the conditions necessary to change states.

We can represent this with a state graph.

<img src="https://raw.githubusercontent.com/byuccl/digital_design_colab/main/State_Machine/media/state_machine.drawio.svg"
width="500" height="300" style="display: block; margin: 0 auto " />
<br>

Each circle on the graph is a state that the machine can be in. Each arrow is a possible transition that can be made from one state to another. The label for each arrow is the variable which the transition is dependent upon. If the machine is in "Entering a choice" it can either move to "Waiting for Payment" or "IDLE" depending on which signal it recieves. By representing your design with a state graph, it is easy to visualize it's functionality and also find places where potential problems could arise. Additionally, it is easy to take the next step and move from a state graph to coding the hardware implementation in SystemVerilog.

### Outputs

Another thing that is useful to include in a state machine is outputs. Outputs, also called actions, are how the machine interacts with the world around it. For example, you could have a yellow LED light up while it is in the "Waiting for Payment" state, and a green LED light up while it is in the "Dispensing Food" state. Outputs also include everything the state machine does, including dispensing the food. But some outputs shouldn't be only dependent on the state. You could have a noise that plays when it switches to the "Entering a choice" state, but you wouldn't want it to keep playing the whole time someone is making their decision. You could make the output dependent on a stop signal from a timer in addition to the state that this is in, or maybe it only occurs on the transition.

The LED outputs mentioned are know as Moore outputs. A Moore output is dependent only on the current state, and ignores all other signals. The sound output is an example of a Mealy output. Mealy outputs are dependent on the current state and another signal. These are often high as the state machine switches states. Because of this, Mealy outputs will be written next to transition lines on the State Graph, and Moore outputs will be written next to the state it is dependent on. 

### Example 2

Think of a washer-dryer combo at a laundromat.

It waits for someone to put in some coins. If coins are inserted, it waits for a button to be pressed to choose the service. If there are enough coins, it can be set to wash, dry or both. It then enters either the wash or dry cycle. If both were selected it will enter dry after washing is done. When dry finishes the machine goes back into idle state.

It has the following states:
1. Idle
2. Waiting for selection.
3. Washing
4. Drying
5. Done


<img src="https://raw.githubusercontent.com/byuccl/digital_design_colab/main/State_Machine/media/washing_machine.drawio.svg"
width="500" height="300" style="display: block; margin: 0 auto " />
<br>

The buttons and coins are used to switch it between the different states.  

Though not everything is listed, this state machine also includes a few Moore outputs. For example, while in the DONE state, the beeping signal is high which drives the beep sound maker. An example of a Mealy Output would be the machine doing what it needs to switch from Washing to Drying.

### Potential Mistakes


There are some things one should be aware of when creating a state machine. There are two cases which can create a bad state machine. They are incomplete and conflicting states. Take look at the transitions out of state A in the following graphs.

<img src="https://raw.githubusercontent.com/byuccl/digital_design_colab/main/State_Machine/media/StateGrapeTypes.drawio.svg"
width="2000" height="340" style="display: block; margin: 0 auto " />
<br>

Incomplete State:
In the second graph, there are certain conditions that will result in nothing happening. This will cause unknown behavior. For example, if both X and Y are low, there is no defined path. To check for Incomplete states, you can create a truth table of all possible values and make sure each has an outcome. An easy solution for coding is to use an else statement that sets the next state as the current state.

Conflicting State:
In this example, the same inputs can result in either state triggering which is undefined behavior. If X and Y are both high, it is told to go to both state B and C. Which one will win out is impossible to tell. This can be fixed by removing overlap and using specific logic.




In [ ]:
#Questions


In [ ]:
#@title  { display-mode: "form" }
#@title  { display-mode: "form" }
#@markdown Use this diagram for the following questions
#@markdown <p align="left"> <img src="https://raw.githubusercontent.com/byuccl/digital_design_colab2/master/Exercises/state_machines/media/state_graph_a.drawio.svg" width="500" height="" style="display: block; margin: 0 auto" /> </p>
#@markdown 
print_ms_grid(1)
print("\n")
print_ms_grid(2)
print("\n")
print_ms_grid(3)
print("\n")


### Coding State Machines

Now lets talk about how to implement a state machine using RTL. 

The state: How will your code know which state you are in? You could assign each state an integer value and store that value, but that can get complicated as it will be hard to tell which state is which.  
You can instead offload this work to the compiler. By using `typedef enum` we can define the states for the state machine.  
Example: `typedef enum logic [1:0] {StateOne, StateTwo, StateThree} statetype;`
    Each state will be translated into a binary value. Make sure your state type is wide enough that each one can be represented.  
    The last word `statetype` is the new name of the type. So if you wanted to make another state variable you would use `statetype MyNewStateType;`  


So now you have the variable storing your state, what should we do with it?  
Well we want the behavior to change depending on the state.  
For this we can use a `switch` statement. Different code will execute depending on the code. 
```
case(myStateType):
    StateOne: begin
    end
    State Two: begin
    end
    StateThree: begin
    end
endcase;
```
We will put this in an `always_comb` block. But now how will we handle updating the state?   
We will use an `always_ff` block along with a `nextState` variable. This is because the `currentState` variable needs to be a flip-flop in order to store the value. In our `always_comb` block we will set `nextState` to the correct value depending on `currentState` and any other factors. Then in the `always_ff` block we will simply set `currentState<=nextState`.  


Example:
```
//There is an input button
typedef enum logic [1:0] {IDLE, ONE_SHOT, WAIT} stateType;
logic pressed;
statetype currentState, nextState;

always_comb begin
    output = 1'b0; //The default case of Output. Prevents latches
    case(currentState)
        IDLE: begin
            if(button) //If the button is pressed, activate the oneshot, else wait
                nextState = ONE_SHOT;
            else
                nextState = WAIT;
        end
        ONE_SHOT: begin //If the button has been pressed, set the output high for one clock cycle
            nextState = WAIT;
            output = 1'b1;
        end
        WAIT: begin //Wait for teh button to be released
            if(button)
                nextState = WAIT;
            else
                nextState = IDLE;
        end
    endcase
end

always_ff @(posedge clk) begin
    currentState <= nextState; //Update the state
end
```
This is a simple One Shot. It takes in a signal as an input (usually a button) and has it only be high for one cycle. This is quite useful for buttons to make sure it only registers one press each time you press it, and not a lot of presses. This is useful for things like a counter, that increments each type you press a button. You don't want a press and hold to register as thousands of presses.







### Questions

In [ ]:
#@title 
#@markdown ```
#@markdown StateOne: begin
#@markdown  if(x)
#@markdown      nextState = StateTwo;
#@markdown  if(y)
#@markdown      nextSTate = StateOne;
#@markdown end
#@markdown ```
print_mc_grid(1)

In [ ]:
#@title 
#@markdown ```
#@markdown StateOne: begin
#@markdown  if(x)
#@markdown      nextState = StateTwo;
#@markdown  else if(y)
#@markdown      nextSTate = StateOne;
#@markdown end
#@markdown ```
print_mc_grid(2)

In [ ]:
#@title 
#@markdown ```
#@markdown StateOne: begin
#@markdown  if(x)
#@markdown      nextState = StateTwo;
#@markdown end
#@markdown ```
print_mc_grid(3)

In [ ]:
#@title 
#@markdown ```
#@markdown StateOne: begin
#@markdown  nextState = StateOne;
#@markdown  if(x)
#@markdown      nextState = StateTwo;
#@markdown end
#@markdown ```
print_mc_grid(4)